# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [1]:
import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from tqdm.notebook import tqdm

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [3]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
# Для начала реализуем сами критерии


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """
    if len(x) == 0:
        return 0.0
    
    counter = Counter(x)
    total = len(x)
    sum_sq_probs = sum((count / total) ** 2 for count in counter.values())
    
    return 1.0 - sum_sq_probs



def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """

    if len(x) == 0:
        return 0.0
    
    counter = Counter(x)
    total = len(x)
    
    entropy = 0.0
    for count in counter.values():
        probability = count / total
        entropy -= probability * np.log2(probability)
    
    return entropy


def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    left_y, right_y = np.asarray(left_y), np.asarray(right_y)
    y = np.concatenate([left_y, right_y])
    R_l, R_r = len(left_y), len(right_y)
    return criterion(y) - (R_l / len(y)) * criterion(left_y) - (R_r / len(y)) * criterion(right_y)

In [5]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [6]:

class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.X = np.asarray(X)
        self.y = np.asarray(y)

        bootstrap_data = self._generate_bootstrap_samples()
        self._boot_X, self._boot_y, self._out_of_bag_X, self._out_of_bag_y = bootstrap_data
        
        self._criterion = gini if criterion == "gini" else entropy
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features if max_features != "auto" else int((X.shape[1])**0.5)
        
        self._root = self._build_node(self._boot_X, self._boot_y, 0)

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y
    
    def _generate_bootstrap_samples(self):
        n_samples = self.X.shape[0]
        bootstrap_indices = np.random.choice(n_samples, size=n_samples, replace=True)
        bootstrap_X = self.X[bootstrap_indices]
        bootstrap_y = self.y[bootstrap_indices]
        
        all_indices = set(range(n_samples))
        bootstrap_unique_indices = set(bootstrap_indices)
        oob_indices = list(all_indices - bootstrap_unique_indices)
        
        oob_X = self.X[oob_indices] if oob_indices else np.array([])
        oob_y = self.y[oob_indices] if oob_indices else np.array([])
        
        return bootstrap_X, bootstrap_y, oob_X, oob_y
    

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        _max_depth = self._max_depth is not None and depth >= self._max_depth
        _max_samples = len(classes) < 2 * self._min_samples_leaf
        _same_classes = len(np.unique(classes)) == 1

        if _max_depth or _max_samples or _same_classes:
            return DecisionTreeLeaf(classes)

        best_split = self._find_split(points, classes)
        
        if best_split is None:
            return DecisionTreeLeaf(classes)
        
        split_dim, left_mask, right_mask = best_split
        

        left_node = self._build_node(points[left_mask], classes[left_mask], depth + 1)
        right_node = self._build_node(points[right_mask], classes[right_mask], depth + 1)
        
        return DecisionTreeInternalNode(
            split_dim=split_dim,
            left=left_node,
            right=right_node
        )
    
    def _find_split(self, points, classes):
        _, n_features = points.shape
        best_gain = -np.inf
        best_split = None

        
        
        trying_features = np.random.choice(
            n_features, 
            size=min(self._max_features, n_features), 
            replace=False
        )
        
        for feature_idx in trying_features:
            feature_values = points[:, feature_idx]
            unique_values = np.unique(feature_values)
            
            if len(unique_values) <= 1:
                continue

            left_mask = feature_values == 0
            right_mask = ~left_mask
            
            if (np.sum(left_mask) < self._min_samples_leaf or 
                np.sum(right_mask) < self._min_samples_leaf):
                continue
            
            current_gain = gain(classes[left_mask], classes[right_mask], self._criterion)
            
            if current_gain > best_gain:
                best_gain = current_gain
                best_split = (feature_idx, left_mask, right_mask)
        
        return best_split
    
    def _predict(self, points: np.ndarray, node: DecisionTreeNode) -> np.ndarray:
        if isinstance(node, DecisionTreeLeaf):
            return np.full(points.shape[0], node.max_class)
        
        left_mask = points[:, node.split_dim] == 0
        right_mask = ~left_mask
        predictions = np.empty(points.shape[0], dtype=self.y.dtype)
    
        if np.any(left_mask):
            predictions[left_mask] = self._predict(points[left_mask], node.left)
        
        if np.any(right_mask):
            predictions[right_mask] = self._predict(points[right_mask], node.right)
        
        return predictions

    def predict(self, points: np.ndarray) -> np.ndarray:
        return self._predict(points, self._root)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [7]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10, random_state=21):
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []
        self._random_state = random_state

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self._n_features = X.shape[1]
        
        self._estimators = []
        for _ in range(self._n_estimators):
            tree = DecisionTree(
                X=X,
                y=y,
                criterion=self._criterion,
                max_depth=self._max_depth,
                min_samples_leaf=self._min_samples_leaf,
                max_features=self._max_features
            )
            self._estimators.append(tree)

    def predict(self, X):
        X = np.asarray(X)
        if len(self._estimators) == 0:
            raise RuntimeError("Model not fitted yet")
        
        all_predictions = []
        for tree in self._estimators:
            pred = tree.predict(X)
            all_predictions.append(pred)

        predictions_matrix = np.column_stack(all_predictions)
        final_predictions = []
        for i in range(predictions_matrix.shape[0]):
            row = predictions_matrix[i, :]
            unique, counts = np.unique(row, return_counts=True)
            most_common = unique[counts.argmax()]
            final_predictions.append(most_common)
        
        return np.array(final_predictions)
    
    def get_params(self, deep=True):
        return {
            'criterion': self._criterion,
            'max_depth': self._max_depth,
            'min_samples_leaf': self._min_samples_leaf,
            'max_features': self._max_features,
            'n_estimators': self._n_estimators,
            'random_state': self._random_state
        }

    def set_params(self, **params):
        for key, value in params.items():
            if key == 'criterion':
                self._criterion = value
            elif key == 'max_depth':
                self._max_depth = value
            elif key == 'min_samples_leaf':
                self._min_samples_leaf = value
            elif key == 'max_features':
                self._max_features = value
            elif key == 'n_estimators':
                self._n_estimators = value
            elif key == 'random_state':
                self._random_state = value
        return self

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [8]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc: RandomForestClassifier) -> np.ndarray:
    n_features = rfc.n_features
    importance = np.zeros(n_features)

    err_oob_list = []
    for tree in rfc.estimators:
        oob_X, oob_y = tree.out_of_bag
        if len(oob_y) > 0:
            pred = tree.predict(oob_X)
            err_oob = 1 - accuracy_score(oob_y, pred)
            err_oob_list.append(err_oob)
        else:
            err_oob_list.append(0.0)
    
    for feature_idx in range(n_features):
        feature_importances = []
        
        for tree, err_oob in zip(rfc.estimators, err_oob_list):
            oob_X, oob_y = tree.out_of_bag
            
            if len(oob_y) == 0:
                continue
            
            shuffled_X = oob_X.copy()
            np.random.shuffle(shuffled_X[:, feature_idx])
            
            pred_shuffled = tree.predict(shuffled_X)
            err_oob_j = 1 - accuracy_score(oob_y, pred_shuffled)
            
            feature_importances.append(err_oob_j - err_oob)
        importance[feature_idx] = np.mean(feature_importances) if feature_importances else 0.0
    
    return importance


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [9]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-1.05920915e-03 -6.14926864e-04  1.72753287e-01  1.63075028e-01
  3.34761856e-01 -1.30265256e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [10]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [11]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9, random_state=21)

In [12]:
def find_best_params(X_train, y_train):
    rfc = RandomForestClassifier()
    param_grid = {
        'n_estimators': [10, 20, 30],
        'max_depth': [5, 10, 20],
        'min_samples_leaf': [1, 3, 5],
        'max_features': [3, 10,'auto'],
        'criterion': ['gini', 'entropy']
    }

    grid = GridSearchCV(
        estimator=rfc,
        param_grid=param_grid,  
        scoring='accuracy',
        cv=3,
        n_jobs=-1,
        verbose=1 
    )

    grid.fit(X_train, y_train)
    print("Лучшие параметры:", grid.best_params_)
    print("Accuracy:", grid.best_score_)

    return grid.best_params_

In [13]:
best_params = find_best_params(X_train, y_age_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Лучшие параметры: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 30}
Accuracy: 0.698920207544524


#### Возраст

In [14]:
rfc = RandomForestClassifier(**best_params, random_state=21)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7011349306431274
Most important features:
1. mudakoff
2. rhymes
3. ovsyanochan
4. styd.pozor
5. 4ch
6. dayvinchik
7. rapnewrap
8. pravdashowtop
9. iwantyou
10. tumblr_vacuum
11. pixel_stickers
12. reflexia_our_feelings
13. bot_maxim
14. memeboizz
15. leprum
16. ne1party
17. pozor
18. ultrapir
19. xfilm
20. thesmolny


In [15]:
best_params = find_best_params(X_train, y_sex_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Лучшие параметры: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 20}
Accuracy: 0.8437806759220305


#### Пол

In [16]:
rfc = RandomForestClassifier(**best_params, random_state=21)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8461538461538461
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. 4ch
6. 9o_6o_9o
7. zerofat
8. be.women
9. be.beauty
10. femalemem
11. cook_good
12. bon
13. i_d_t
14. rapnewrap
15. recipes40kg
16. woman.blog
17. reflexia_our_feelings
18. sh.cook
19. beauty
20. thesmolny


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [17]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(
    iterations=100,           
    depth=7,    
    learning_rate=0.05,        
    loss_function='MultiClass',
    verbose=False,
    random_state=21,
)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

Accuracy: 1.0
Importance: [3.71769128e-03 4.63054971e-03 2.79505147e+01 2.79623985e+01
 4.40755371e+01 3.20145165e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [18]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [19]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score = 0
    best_params = None
    
    max_depth = [4, 6, 8] 
    min_samples_leaf = [1, 3, 5] 
    learning_rate = [0.01, 0.1, 0.3] 
    
    is_multiclass = len(np.unique(y_train)) > 2
    loss_function = 'MultiClass' if is_multiclass else 'Logloss'
    
    for lr in learning_rate:
        for depth in max_depth:
            for min_samples in min_samples_leaf:
                model = CatBoostClassifier(
                    iterations=100,
                    depth=depth,
                    learning_rate=lr,
                    min_data_in_leaf=min_samples,
                    loss_function=loss_function,
                    verbose=False,
                    random_state=21
                )
                
                model.fit(X_train, y_train, eval_set=(X_eval, y_eval), verbose=False)
                score = model.score(X_eval, y_eval)
                
                if score > best_score:
                    best_score = score
                    best_params = {
                        'learning_rate': lr,
                        'depth': depth, 
                        'min_data_in_leaf': min_samples
                    }
                print(f"lr={lr}, depth={depth}, leaf={min_samples}, score={score:.4f}")
                        
    
    return best_params, best_score

#### Возраст

In [20]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

lr=0.01, depth=4, leaf=1, score=0.6300
lr=0.01, depth=4, leaf=3, score=0.6300
lr=0.01, depth=4, leaf=5, score=0.6300
lr=0.01, depth=6, leaf=1, score=0.6552
lr=0.01, depth=6, leaf=3, score=0.6552
lr=0.01, depth=6, leaf=5, score=0.6552
lr=0.01, depth=8, leaf=1, score=0.6678
lr=0.01, depth=8, leaf=3, score=0.6678
lr=0.01, depth=8, leaf=5, score=0.6678
lr=0.1, depth=4, leaf=1, score=0.7064
lr=0.1, depth=4, leaf=3, score=0.7064
lr=0.1, depth=4, leaf=5, score=0.7064
lr=0.1, depth=6, leaf=1, score=0.7141
lr=0.1, depth=6, leaf=3, score=0.7141
lr=0.1, depth=6, leaf=5, score=0.7141
lr=0.1, depth=8, leaf=1, score=0.7302
lr=0.1, depth=8, leaf=3, score=0.7302
lr=0.1, depth=8, leaf=5, score=0.7302
lr=0.3, depth=4, leaf=1, score=0.7127
lr=0.3, depth=4, leaf=3, score=0.7127
lr=0.3, depth=4, leaf=5, score=0.7127
lr=0.3, depth=6, leaf=1, score=0.7330
lr=0.3, depth=6, leaf=3, score=0.7330
lr=0.3, depth=6, leaf=5, score=0.7330
lr=0.3, depth=8, leaf=1, score=0.7246
lr=0.3, depth=8, leaf=3, score=0.7246
lr=

({'learning_rate': 0.3, 'depth': 6, 'min_data_in_leaf': 1},
 np.float64(0.7330063069376314))

In [21]:
cb_model = CatBoostClassifier(iterations=100,                 
    loss_function='MultiClass',
    verbose=False,
    random_state=21,
    **best_params
)

cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.712484237074401
Most important features:
1. ovsyanochan
2. mudakoff
3. styd.pozor
4. rhymes
5. dayvinchik
6. leprum
7. 4ch
8. xfilm
9. bot_maxim
10. rapnewrap


#### Пол

In [22]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

lr=0.01, depth=4, leaf=1, score=0.8059
lr=0.01, depth=4, leaf=3, score=0.8059
lr=0.01, depth=4, leaf=5, score=0.8059
lr=0.01, depth=6, leaf=1, score=0.8374
lr=0.01, depth=6, leaf=3, score=0.8374
lr=0.01, depth=6, leaf=5, score=0.8374
lr=0.01, depth=8, leaf=1, score=0.8472
lr=0.01, depth=8, leaf=3, score=0.8472
lr=0.01, depth=8, leaf=5, score=0.8472
lr=0.1, depth=4, leaf=1, score=0.8662
lr=0.1, depth=4, leaf=3, score=0.8662
lr=0.1, depth=4, leaf=5, score=0.8662
lr=0.1, depth=6, leaf=1, score=0.8753
lr=0.1, depth=6, leaf=3, score=0.8753
lr=0.1, depth=6, leaf=5, score=0.8753
lr=0.1, depth=8, leaf=1, score=0.8767
lr=0.1, depth=8, leaf=3, score=0.8767
lr=0.1, depth=8, leaf=5, score=0.8767
lr=0.3, depth=4, leaf=1, score=0.8774
lr=0.3, depth=4, leaf=3, score=0.8774
lr=0.3, depth=4, leaf=5, score=0.8774
lr=0.3, depth=6, leaf=1, score=0.8739
lr=0.3, depth=6, leaf=3, score=0.8739
lr=0.3, depth=6, leaf=5, score=0.8739
lr=0.3, depth=8, leaf=1, score=0.8711
lr=0.3, depth=8, leaf=3, score=0.8711
lr=

({'learning_rate': 0.3, 'depth': 4, 'min_data_in_leaf': 1},
 np.float64(0.877365101611773))

In [23]:
cb_model = CatBoostClassifier(iterations=100,                 
    loss_function='MultiClass',
    verbose=False,
    random_state=21,
    **best_params
)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8675914249684742
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. mudakoff
5. academyofman
6. i_d_t
7. thesmolny
8. femalemem
9. igm
10. zerofat
